In [5]:
print("Hello world")

# String concatenation
first = "Hello "
second = "World"
long_result = first + second + "!"
print(long_result)

Hello world
Hello World!


In [7]:
import pandas as pd
from google.colab import data_table as dt
dt.enable_dataframe_formatter()
dt.disable_dataframe_formatter()
airline_passengers = pd.read_csv('/content/drive/MyDrive/airline-passengers.csv')
airline_passengers

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [8]:
airline_passengers.shape
airline_passengers.describe()
airline_passengers.values

array([['1949-01', 112],
       ['1949-02', 118],
       ['1949-03', 132],
       ['1949-04', 129],
       ['1949-05', 121],
       ['1949-06', 135],
       ['1949-07', 148],
       ['1949-08', 148],
       ['1949-09', 136],
       ['1949-10', 119],
       ['1949-11', 104],
       ['1949-12', 118],
       ['1950-01', 115],
       ['1950-02', 126],
       ['1950-03', 141],
       ['1950-04', 135],
       ['1950-05', 125],
       ['1950-06', 149],
       ['1950-07', 170],
       ['1950-08', 170],
       ['1950-09', 158],
       ['1950-10', 133],
       ['1950-11', 114],
       ['1950-12', 140],
       ['1951-01', 145],
       ['1951-02', 150],
       ['1951-03', 178],
       ['1951-04', 163],
       ['1951-05', 172],
       ['1951-06', 178],
       ['1951-07', 199],
       ['1951-08', 199],
       ['1951-09', 184],
       ['1951-10', 162],
       ['1951-11', 146],
       ['1951-12', 166],
       ['1952-01', 171],
       ['1952-02', 180],
       ['1952-03', 193],
       ['1952-04', 181],


Data Preperation for univariate Analysis considering as a CNN model

In [9]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

Making use of sequence

In [10]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


Select Subset of Columns

In [11]:
import pandas as pd

data = pd.read_csv (r'C:\Users\Ron\Desktop\Clients.csv')   
df = pd.DataFrame(data, columns= ['Person Name','Country'])
print (df)

FileNotFoundError: ignored

Encoding Time-Series into Images for Forecasting using Convolutional Neural Networks


**Gramian Angular Fields (GAF)** are images representing a timeseries in the Polar Ordinate system (i.e. each point on the plane is determined by a distance from a reference point and an angle from a reference direction). Thus each GAF represents a temporal correlation between each time point.



In [ ]:
import pandas as pd
import os
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
PATH = os.path.join(os.path.dirname(__file__), 'TimeSeries')

def data_to_image_preprocess():
    """
    :return: None
    """
    ive_data = 'IVE_tickbidask.txt'
    col_name = ['Date', 'Time', 'Open', 'High', 'Low', 'Volume']
    df = pd.read_csv(os.path.join(PATH, ive_data), names=col_name, header=None)
    # Drop unnecessary data
    df = df.drop(['High', 'Low', 'Volume'], axis=1)
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], infer_datetime_format=True)
    df = df.groupby(pd.Grouper(key='DateTime', freq='1h')).mean().reset_index()
    df['Open'] = df['Open'].replace(to_replace=0, method='ffill')
    # Remove non trading days and times
    clean_df = clean_non_trading_times(df)
    # Send to slicing
    set_gaf_data(clean_df)

NameError: ignored

In [ ]:
def clean_non_trading_times(df):
    """
    :param df: Data with weekends and holidays
    :return trading_data:
    """
    # Weekends go out
    df = df[df['DateTime'].dt.weekday < 5].reset_index(drop=True)
    df = df.set_index('DateTime')
    # Remove non trading hours
    df = df.between_time('9:00','16:00')
    df.reset_index(inplace=True)
    # Holiday days we want to delete from data
    holidays = calendar().holidays(start='2000-01-01', end='2020-12-31')
    m = df['DateTime'].isin(holidays)
    clean_df = df[~m].copy()
    trading_data = clean_df.fillna(method='ffill')
    return trading_data

In [ ]:
def set_gaf_data(df):
    """
    :param df: DataFrame data
    :return: None
    """
    dates = df['DateTime'].dt.date
    dates = dates.drop_duplicates()
    list_dates = dates.apply(str).tolist()
    index = 20 # rows of data used on each GAF
    # Container to store data for the creation of GAF
    decision_map = {key: [] for key in ['LONG', 'SHORT']}
    while True:
        if index >= len(list_dates) - 1:
            break
        # Select appropriate timeframe
        data_slice = df.loc[(df['DateTime'] > list_dates[index - 20]) & (df['DateTime'] < list_dates[index])]
        gafs = []
        # Group data_slice by time frequency
        for freq in ['1h', '2h', '4h', '1d']:
            group_dt = data_slice.groupby(pd.Grouper(key='DateTime', freq=freq)).mean().reset_index()
            group_dt = group_dt.dropna()
            gafs.append(group_dt['Close'].tail(20))
        # Decide what trading position we should take on that day
        future_value = df[df['DateTime'].dt.date.astype(str) == list_dates[index]]['Close'].iloc[-1]
        current_value = data_slice['Close'].iloc[-1]
        decision = trading_action(future_close=future_value, current_close=current_value)
        decision_map[decision].append([list_dates[index - 1], gafs])
        index += 1

In [ ]:
def trading_action(data, index):
    """
    :param data: DataFrame 
    :param index: Date Index for slicing
    :return: Folder destination as String
    """
    future_open = data[data['DateTime'].dt.date.astype(str) == index]['Open'].iloc[0]
    future_close = data[data['DateTime'].dt.date.astype(str) == index]['Open'].iloc[-1]
    if future_open < future_close:
        decision = 'LONG'
    else:
        decision = 'SHORT'
    return decision

In [ ]:
def create_gaf(ts):
    """
    :param ts:
    :return:
    """
    data = dict()
    gadf = GramianAngularField(method='difference', image_size=ts.shape[0])
    data['gadf'] = gadf.fit_transform(pd.DataFrame(ts).T)[0] # ts.T)
    return data

**THE MODEL**

In [ ]:
cnn_networks = 3
model = []
for j in range(cnn_networks):
    model.append(
        tf.keras.models.Sequential([
            #  First Convolution
            Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(255, 255, 3)),
            BatchNormalization(),
            Conv2D(32, kernel_size=(3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(32, kernel_size=(3, 3), strides=2, padding='same', activation='relu'),
            BatchNormalization(),
            Dropout(0.4),
            # Second Convolution
            Conv2D(64, kernel_size=(3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(64, kernel_size=(3, 3), activation='relu'),
            BatchNormalization(),
            Conv2D(64, kernel_size=(3, 3), strides=2, padding='same', activation='relu'),
            BatchNormalization(),
            Dropout(0.4),
            # Third Convolution
            Conv2D(128, kernel_size=4, activation='relu'),
            BatchNormalization(),
            Flatten(),
            Dropout(0.4),
            # Output layer
            Dense(1, activation='sigmoid')]
        ))
    # Compile each model
    model[j].compile(optimizer=Adam(lr=LR), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# All images will be rescaled by 1./255
train_validate_datagen = ImageDataGenerator(rescale=1/255, validation_split=SPLIT)  # set validation split
test_datagen = ImageDataGenerator(rescale=1/255)
data_chunks = ensemble_data(cnn_networks, IMAGES_PATH)
for j in range(cnn_networks):
    print('Net : {}'.format(j+1))
    df_train = data_chunks[j].iloc[:-60]
    df_test = data_chunks[j].iloc[-60:]
    train_generator = train_validate_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_PATH,
        target_size=(255, 255),
        x_col='Images',
        y_col='Labels',
        batch_size=32,
        class_mode='binary',
        subset='training')

    validation_generator = train_validate_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_PATH,
        target_size=(255, 255),
        x_col='Images',
        y_col='Labels',
        batch_size=32,
        class_mode='binary',
        subset='validation')

    test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='Images',
        y_col='Labels',
        directory=IMAGES_PATH,
        target_size=(255, 255),
        class_mode='binary')

In [ ]:
steps_per_epoch = train_generator.n // train_generator.batch_size
        validation_steps = validation_generator.n // validation_generator.batch_size
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=0, factor=0.5, min_lr=0.00001)
        history = model[j].fit_generator(train_generator,
                                         epochs=EPOCHS,
                                         steps_per_epoch=steps_per_epoch,
                                         validation_data=validation_generator,
                                         callbacks=[learning_rate_reduction],
                                         verbose=0)
        print('CNN Model {0:d}: '
              'Epochs={1:d}, '
              'Training Accuracy={2:.5f}, '
              'Validation Accuracy={3:.5f}'.format(j + 1,
                                                   EPOCHS,
                                                   max(history.history['acc']),
                                                   max(history.history['val_acc'])))

        scores = model[j].evaluate_generator(test_generator, steps=5)
        print("{0}s: {1:.2f}%".format(model[j].metrics_names[1], scores[1]*100))
        string_list = []
        model[j].summary(print_fn=lambda x: string_list.append(x))
        summary = "\n".join(string_list)
        logging = ['{0}: {1}'.format(key, val[-1]) for key, val in history.history.items()]
        log = 'Results:\n' + '\n'.join(logging)
        model[j].save(os.path.join(REPO, 'computer_vision_model_{0}_of_{1}_{2}.h5'.format(TIMESTAMP, j, cnn_networks)))
        f = open(os.path.join(REPO, 'computer_vision_summary_{0}_of_{1}_{2}.h5'.format(TIMESTAMP, j, cnn_networks)), 'w')
        f.write("EPOCHS: {0}\nSteps per epoch: {1}\nValidation steps: {2}\nVal Split:{3}\nLearning RT:{5}\n\n\n{4}"
                "\n\n=========TRAINING LOG========\n{6}".format(EPOCHS, steps_per_epoch, validation_steps,  SPLIT, summary,
                                                                LR, log))
        f.close()steps_per_epoch = train_generator.n // train_generator.batch_size
        validation_steps = validation_generator.n // validation_generator.batch_size
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=0, factor=0.5, min_lr=0.00001)
        history = model[j].fit_generator(train_generator,
                                         epochs=EPOCHS,
                                         steps_per_epoch=steps_per_epoch,
                                         validation_data=validation_generator,
                                         callbacks=[learning_rate_reduction],
                                         verbose=0)
        print('CNN Model {0:d}: '
              'Epochs={1:d}, '
              'Training Accuracy={2:.5f}, '
              'Validation Accuracy={3:.5f}'.format(j + 1,
                                                   EPOCHS,
                                                   max(history.history['acc']),
                                                   max(history.history['val_acc'])))

        scores = model[j].evaluate_generator(test_generator, steps=5)
        print("{0}s: {1:.2f}%".format(model[j].metrics_names[1], scores[1]*100))
        string_list = []
        model[j].summary(print_fn=lambda x: string_list.append(x))
        summary = "\n".join(string_list)
        logging = ['{0}: {1}'.format(key, val[-1]) for key, val in history.history.items()]
        log = 'Results:\n' + '\n'.join(logging)
        model[j].save(os.path.join(REPO, 'computer_vision_model_{0}_of_{1}_{2}.h5'.format(TIMESTAMP, j, cnn_networks)))
        f = open(os.path.join(REPO, 'computer_vision_summary_{0}_of_{1}_{2}.h5'.format(TIMESTAMP, j, cnn_networks)), 'w')
        f.write("EPOCHS: {0}\nSteps per epoch: {1}\nValidation steps: {2}\nVal Split:{3}\nLearning RT:{5}\n\n\n{4}"
                "\n\n=========TRAINING LOG========\n{6}".format(EPOCHS, steps_per_epoch, validation_steps,  SPLIT, summary,
                                                                LR, log))
        f.close()

**Recurrence Plot**


In [ ]:
from pyts.datasets import load_gunpoint
from pyts.image import RecurrencePlot
X, _, _, _ = load_gunpoint(return_X_y=True)
transformer = RecurrencePlot()
X_new = transformer.transform(X)
X_new.shape

ModuleNotFoundError: ignored

Images can be flattened by setting the flatten parameter to True, so that classification can be directly performed:

In [ ]:
from pyts.image import RecurrencePlot
from pyts.datasets import load_gunpoint
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = load_gunpoint(return_X_y=True)
recurrence = RecurrencePlot(dimension=15, time_delay=3, flatten=True)
logistic = LogisticRegression(solver='liblinear')
clf = make_pipeline(recurrence, logistic)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)


Gramian Angular Field
{making use of pyts)

In [ ]:
from pyts.datasets import load_gunpoint
from pyts.image import GramianAngularField
X, _, _, _ = load_gunpoint(return_X_y=True)
transformer = GramianAngularField()
X_new = transformer.transform(X)
X_new.shape

Images can be flattened by setting the flatten parameter to True, so that classification can be directly performed:

In [ ]:
from pyts.image import GramianAngularField
from pyts.datasets import load_gunpoint
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = load_gunpoint(return_X_y=True)
gaf = GramianAngularField(flatten=True)
logistic = LogisticRegression(solver='liblinear')
clf = make_pipeline(gaf, logistic)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)